# Quantum Phase Estimation (QPE) on a VQE ground-state ansatz (Qiskit)

This notebook now computes and substitutes hardcoded values for:

- `H_qubit`: `qiskit.quantum_info.SparsePauliOp`
- `ansatz`: hardcoded ansatz adapter with `assign_parameters(...) -> QuantumCircuit`
- `theta_star`: hardcoded-VQE optimal parameters
- `E_vqe`: hardcoded-VQE minimum energy

This notebook adds:

1) A phase-only eigenstate check via time-evolution overlap, plus standard QPE to recover the eigenphase.
2) Hamiltonian QPE (scaled phase estimation) to recover the ground-state energy and compare to VQE.


In [1]:
# --- Imports (Qiskit 2.x + qiskit_algorithms 0.4.x style) ---

import numpy as np

from qiskit import QuantumCircuit
from qiskit.quantum_info import SparsePauliOp, Statevector

from qiskit.circuit.library import PauliEvolutionGate
from qiskit.synthesis import SuzukiTrotter

# Sampler primitives (V2) for QPE / HPE
from qiskit.primitives import StatevectorSampler
from qiskit.transpiler import generate_preset_pass_manager

try:
    from qiskit_aer.primitives import SamplerV2 as AerSampler
    from qiskit_aer import AerSimulator
    _HAS_AER = True
except Exception:
    AerSampler = None
    AerSimulator = None
    _HAS_AER = False

# Phase estimation algorithms (external package, per Qiskit migration)
from qiskit_algorithms import PhaseEstimation, HamiltonianPhaseEstimation
from qiskit_algorithms import PhaseEstimationScale


# --- Helpers ---

def bind_ansatz(ansatz: QuantumCircuit, theta_star):
    '''Return ansatz with parameters assigned.

    theta_star can be:
      - dict: {Parameter: value}
      - list/np.ndarray: ordered values matching ansatz.parameters
    '''
    return ansatz.assign_parameters(theta_star, inplace=False)


def pauli_sum_bound(H: SparsePauliOp) -> float:
    '''A simple (generally loose) eigenvalue bound: sum of abs(coeffs).

    Matches PhaseEstimationScale.from_pauli_sum logic for SparsePauliOp.
    '''
    return float(np.sum(np.abs(H.coeffs)))


def build_time_evolution_unitary(
    H: SparsePauliOp,
    time: float,
    trotter_reps: int = 1,
    trotter_order: int = 2,
) -> QuantumCircuit:
    '''Circuit for U(time) = exp(-i * time * H), via PauliEvolutionGate + Suzuki-Trotter.'''
    synthesis = SuzukiTrotter(order=trotter_order, reps=trotter_reps)
    evo_gate = PauliEvolutionGate(H, time=time, synthesis=synthesis)

    U = QuantumCircuit(H.num_qubits)
    U.append(evo_gate, U.qubits)

    # Optional: decompose high-level evolution into basis (useful for transpilation/backends)
    return U.decompose().decompose()


def overlap_and_fidelity(psi: Statevector, phi: Statevector):
    '''Return overlap <psi|phi>, its |.|^2, and its phase angle.'''
    amp = np.vdot(psi.data, phi.data)      # <psi|phi>
    fid = float(np.abs(amp) ** 2)          # |<psi|phi>|^2
    ang = float(np.angle(amp))             # arg(<psi|phi>)
    return amp, fid, ang


In [2]:
# --- Hardcoded substitutions for H_qubit, ansatz, theta_star, E_vqe ---

import importlib.util
import json
import sys
import types
from pathlib import Path

import numpy as np
from qiskit import QuantumCircuit
from qiskit.quantum_info import SparsePauliOp


def _candidate_project_roots():
    cwd = Path.cwd().resolve()
    return [cwd, *cwd.parents]


def _bootstrap_pydephasing_quantum_alias() -> Path:
    # Resolve local quantum module regardless of notebook launch directory.
    quantum_dir = None
    for root in _candidate_project_roots():
        for cand in (root / 'pydephasing' / 'quantum', root / 'quantum'):
            if (cand / 'pauli_polynomial_class.py').exists() and (cand / 'hubbard_latex_python_pairs.py').exists():
                quantum_dir = cand.resolve()
                break
        if quantum_dir is not None:
            break
    if quantum_dir is None:
        raise FileNotFoundError(
            f'Could not locate local quantum module directory from cwd={Path.cwd()}.'
        )

    if 'pydephasing' not in sys.modules:
        root_pkg = types.ModuleType('pydephasing')
        root_pkg.__path__ = [str(quantum_dir.parent)]
        sys.modules['pydephasing'] = root_pkg

    quantum_pkg = types.ModuleType('pydephasing.quantum')
    quantum_pkg.__path__ = [str(quantum_dir)]
    sys.modules['pydephasing.quantum'] = quantum_pkg

    load_order = [
        'pauli_letters_module',
        'pauli_words',
        'pauli_polynomial_class',
        'hartree_fock_reference_state',
        'hubbard_latex_python_pairs',
    ]

    for name in load_order:
        full_name = f'pydephasing.quantum.{name}'
        if full_name in sys.modules:
            continue
        file_path = quantum_dir / f'{name}.py'
        spec = importlib.util.spec_from_file_location(full_name, file_path)
        if spec is None or spec.loader is None:
            raise RuntimeError(f'Failed to load module spec for {full_name} from {file_path}')
        module = importlib.util.module_from_spec(spec)
        sys.modules[full_name] = module
        spec.loader.exec_module(module)

    return quantum_dir


def _resolve_hardcoded_vqe_notebook(quantum_dir: Path) -> Path:
    candidates = [quantum_dir / 'vqe_latex_python_pairs.ipynb']
    for root in _candidate_project_roots():
        candidates.extend([
            root / 'quantum' / 'vqe_latex_python_pairs.ipynb',
            root / 'pydephasing' / 'quantum' / 'vqe_latex_python_pairs.ipynb',
            root / 'vqe_latex_python_pairs.ipynb',
        ])
    for cand in candidates:
        if cand.exists():
            return cand.resolve()
    raise FileNotFoundError('Could not locate vqe_latex_python_pairs.ipynb for hardcoded VQE setup.')


def _load_hardcoded_vqe_namespace(vqe_nb_path: Path):
    payload = json.loads(vqe_nb_path.read_text())
    ns = {}
    for cell in payload.get('cells', []):
        if cell.get('cell_type') != 'code':
            continue
        src = ''.join(cell.get('source', []))
        # Skip report/benchmark driver cells; keep reusable function/class definitions.
        if 'Final benchmark: hardcoded VQE vs Qiskit VQE' in src:
            continue
        exec(src, ns)
    return ns


def _pauli_polynomial_to_sparse_pauli_op(H, tol: float = 1e-12) -> SparsePauliOp:
    terms = []
    for term in H.return_polynomial():
        coeff = complex(term.p_coeff)
        if abs(coeff) < tol:
            continue
        label = term.pw2strng().replace('e', 'I').upper()
        terms.append((label, coeff))
    if not terms:
        nq = int(H.return_polynomial()[0].nqubit()) if H.return_polynomial() else 1
        terms = [('I' * nq, 0.0)]
    return SparsePauliOp.from_list(terms).simplify(atol=tol)


class _HardcodedAnsatzAdapter:
    """
    Adapter exposing assign_parameters(...) so the existing QPE notebook flow can use
    the hardcoded state-preparation logic directly.
    """

    def __init__(self, hardcoded_ansatz, psi_ref: np.ndarray):
        self._hardcoded_ansatz = hardcoded_ansatz
        self._psi_ref = np.asarray(psi_ref, dtype=complex)
        self._num_parameters = int(getattr(hardcoded_ansatz, 'num_parameters', 0))
        self._nq = int(round(np.log2(self._psi_ref.size)))

    def _coerce_theta(self, theta_input) -> np.ndarray:
        if isinstance(theta_input, dict):
            # Allow empty dict for compatibility with generic assign_parameters APIs.
            if len(theta_input) == 0:
                return np.zeros(self._num_parameters, dtype=float)
            raise ValueError('Hardcoded ansatz adapter expects array-like theta or empty dict.')

        theta = np.asarray(theta_input, dtype=float).reshape(-1)
        if theta.size != self._num_parameters:
            raise ValueError(
                f'theta size mismatch: expected {self._num_parameters}, got {theta.size}'
            )
        return theta

    def assign_parameters(self, theta_input, inplace: bool = False) -> QuantumCircuit:
        theta = self._coerce_theta(theta_input)
        psi = self._hardcoded_ansatz.prepare_state(theta, self._psi_ref)
        norm = float(np.linalg.norm(psi))
        if norm <= 0.0:
            raise ValueError('Prepared state has zero norm.')
        psi = np.asarray(psi, dtype=complex) / norm

        qc = QuantumCircuit(self._nq, name='hardcoded_hva_state')
        qc.initialize(psi, qc.qubits)
        return qc


_quantum_dir = _bootstrap_pydephasing_quantum_alias()
from pydephasing.quantum.hubbard_latex_python_pairs import build_hubbard_hamiltonian

_vqe_ns = _load_hardcoded_vqe_namespace(_resolve_hardcoded_vqe_notebook(_quantum_dir))

HARDCODED_CFG = {
    'L': 2,
    't': 1.0,
    'U': 4.0,
    'indexing': 'blocked',
    'pbc': True,
    'reps': 2,
    'restarts': 3,
    'seed': 7,
    'maxiter': 1800,
}

L = int(HARDCODED_CFG['L'])
num_particles = _vqe_ns['half_filled_num_particles'](L)

H_hardcoded = build_hubbard_hamiltonian(
    dims=L,
    t=float(HARDCODED_CFG['t']),
    U=float(HARDCODED_CFG['U']),
    v=None,
    repr_mode='JW',
    indexing=HARDCODED_CFG['indexing'],
    pbc=HARDCODED_CFG['pbc'],
)

hardcoded_ansatz = _vqe_ns['HubbardTermwiseAnsatz'](
    dims=L,
    t=float(HARDCODED_CFG['t']),
    U=float(HARDCODED_CFG['U']),
    v=None,
    reps=int(HARDCODED_CFG['reps']),
    repr_mode='JW',
    indexing=HARDCODED_CFG['indexing'],
    pbc=HARDCODED_CFG['pbc'],
    include_potential_terms=True,
)

hf_bits = _vqe_ns['hartree_fock_bitstring'](
    n_sites=L,
    num_particles=num_particles,
    indexing=HARDCODED_CFG['indexing'],
)
psi_ref = _vqe_ns['basis_state'](2 * L, hf_bits)

vqe_hardcoded = _vqe_ns['vqe_minimize'](
    H_hardcoded,
    hardcoded_ansatz,
    psi_ref,
    restarts=int(HARDCODED_CFG['restarts']),
    seed=int(HARDCODED_CFG['seed']),
    maxiter=int(HARDCODED_CFG['maxiter']),
    method='SLSQP',
)

H_qubit_hardcoded = _pauli_polynomial_to_sparse_pauli_op(H_hardcoded)
theta_star_hardcoded = np.asarray(vqe_hardcoded.theta, dtype=float)
E_vqe_hardcoded = float(vqe_hardcoded.energy)
ansatz_hardcoded = _HardcodedAnsatzAdapter(hardcoded_ansatz, psi_ref)

# Substitute notebook-wide symbols.
H_qubit = H_qubit_hardcoded
ansatz = ansatz_hardcoded
theta_star = theta_star_hardcoded
E_vqe = E_vqe_hardcoded

print('Hardcoded substitutions loaded:')
print(f'  module_dir={_quantum_dir}')
print(f'  L={L}, num_qubits={H_qubit.num_qubits}, num_particles={num_particles}')
print(f'  E_vqe (hardcoded) = {E_vqe:.12f}')
print(f'  theta_star length = {theta_star.size}')
print('  ansatz adapter is now injected from hardcoded workflow.')


Hardcoded substitutions loaded:
  module_dir=/Users/jakestrobel/Downloads/dephasing-code-base-main 2/pydephasing/quantum
  L=2, num_qubits=4, num_particles=(1, 1)
  E_vqe (hardcoded) = -0.828427105805
  theta_star length = 8
  ansatz adapter is now injected from hardcoded workflow.


## Segment 1 — “phase-only” time evolution test

Axiomatic time evolution:
\[
U(t)\;\stackrel{\text{axiom}}{:=}\;e^{-iHt}.
\]

With spectral resolution \(H=\sum_j E_j\lvert E_j\rangle\langle E_j\rvert\) and ansatz state \(\lvert\psi\rangle=\sum_j c_j\lvert E_j\rangle\), we get:

\[
\begin{aligned}
U(t)\lvert\psi\rangle
&=e^{-iHt}\sum_j c_j\lvert E_j\rangle \\
&=\sum_j c_j e^{-iHt}\lvert E_j\rangle \\
&=\sum_j c_j e^{-iE_j t}\lvert E_j\rangle, \\
\langle\psi\lvert U(t)\rvert\psi\rangle
&=\sum_{j,k} c_j^*c_k \langle E_j\lvert e^{-iHt}\rvert E_k\rangle \\
&=\sum_{j,k} c_j^*c_k e^{-iE_k t}\langle E_j\lvert E_k\rangle \\
&=\sum_k |c_k|^2 e^{-iE_k t}, \\
F(t)
&:=\big|\langle\psi\lvert U(t)\rvert\psi\rangle\big|^2.
\end{aligned}
\]

If \(\lvert\psi\rangle=\lvert E_{k}\rangle\) is an eigenstate, then \(\langle\psi\lvert U(t)\rvert\psi\rangle=e^{-iE_k t}\) and \(F(t)=1\) (all \(t\)). Deviations of \(F(t)\) from 1 indicate non-eigenstate components.


In [3]:
# --- Segment 1 code: overlap/fidelity eigenstate test ---

# H_qubit, ansatz, theta_star, and E_vqe are injected by the hardcoded setup cell above.

# Bound + evolution time choice (non-wrapping safe choice).
B = pauli_sum_bound(H_qubit)
t = np.pi / B

print(f'Pauli-sum bound B = {B}')
print(f'Chosen evolution time t = pi/B = {t}')

# Prepare |psi> (your VQE ansatz state) as a circuit and a statevector
state_prep = bind_ansatz(ansatz, theta_star)
psi = Statevector.from_instruction(state_prep)

# Build U(t) = exp(-i H t)
U_t = build_time_evolution_unitary(H_qubit, time=t, trotter_reps=1, trotter_order=2)

# Evolve: |psi_t> = U(t)|psi>
evolve_circuit = QuantumCircuit(H_qubit.num_qubits)
evolve_circuit.compose(state_prep, inplace=True)
evolve_circuit.compose(U_t, inplace=True)

psi_t = Statevector.from_instruction(evolve_circuit)

amp, fid, ang = overlap_and_fidelity(psi, psi_t)

print(f'<psi|U(t)|psi> = {amp}')
print(f'F(t) = |<psi|U(t)|psi>|^2 = {fid}')
print(f'arg(<psi|U(t)|psi>) = {ang} rad')

# If fid ~ 1, the state is (approximately) an eigenstate and the evolution is (approximately) phase-only.
# For an eigenstate with energy E, the ideal phase is: ang ≡ -E t  (mod 2π).



Pauli-sum bound B = 10.0
Chosen evolution time t = pi/B = 0.3141592653589793
<psi|U(t)|psi> = (0.9714474320096118+0.22356631400483795j)
F(t) = |<psi|U(t)|psi>|^2 = 0.9936920099157793
arg(<psi|U(t)|psi>) = 0.22619881018233262 rad


## Segment 2 — Standard QPE phase recovery

Axiomatic QPE eigenphase relation:
\[
U\lvert\phi\rangle \;=\; e^{2\pi i \varphi}\lvert\phi\rangle,\qquad \varphi\in[0,1).
\]

Specialize to \(U(t)=e^{-iHt}\) and \(\lvert\phi\rangle=\lvert E\rangle\):
\[
\begin{aligned}
U(t)\lvert E\rangle
&=e^{-iHt}\lvert E\rangle\\
&=e^{-iEt}\lvert E\rangle\\
&=e^{2\pi i\varphi}\lvert E\rangle,\\
2\pi\varphi &\equiv -Et \;(\mathrm{mod}\;2\pi),\\
\varphi &\equiv -\frac{Et}{2\pi}\;(\mathrm{mod}\;1).
\end{aligned}
\]

If we pick a bound \(B\ge \max_j |E_j|\) and set \(t=\pi/B\), then \(-Et\in[-\pi,\pi]\) so the “unwrap” is unique via
\[
\tilde\varphi := \begin{cases}
\varphi, & \varphi\le 1/2,\\
\varphi-1, & \varphi>1/2,
\end{cases}
\qquad
E = -2B\,\tilde\varphi.
\]


In [4]:
# --- Segment 2 code: standard QPE phase recovery for U(t) = exp(-i H t) ---

m = 8               # evaluation register bits (phase resolution ~ 1/2^m)
shots = 4096
seed = 1234

USE_AER = False  # Set True only if qiskit-aer is stable in your environment.

if USE_AER and _HAS_AER:
    backend = AerSimulator()
    pass_manager = generate_preset_pass_manager(optimization_level=3, backend=backend)
    sampler = AerSampler(default_shots=shots, seed=seed)
    print('Using AerSampler backend.')
else:
    pass_manager = None
    sampler = StatevectorSampler(default_shots=shots, seed=seed)
    if USE_AER and not _HAS_AER:
        print('Aer requested but unavailable; using StatevectorSampler fallback.')
    else:
        print('Using StatevectorSampler backend.')

qpe = PhaseEstimation(
    num_evaluation_qubits=m,
    sampler=sampler,
    transpiler=pass_manager,
)

qpe_res = qpe.estimate(unitary=U_t, state_preparation=state_prep)

phi_qpe = float(qpe_res.phase)        # in [0,1)
theta_qpe = 2 * np.pi * phi_qpe       # in [0, 2π)

print(f'QPE most-likely phase (normalized) phi = {phi_qpe}')
print(f'QPE most-likely angle theta = 2π phi = {theta_qpe} rad')

# Compare to overlap-extracted phase angle. Note both are modulo 2π.
# We can map theta_qpe to (-π, π] for an easier comparison.
theta_qpe_pm = ((theta_qpe + np.pi) % (2 * np.pi)) - np.pi
print(f'QPE theta mapped to (-π, π] : {theta_qpe_pm} rad')

print(f'Overlap angle arg(<psi|U(t)|psi>) : {ang} rad')

# Optional: recover energy from phi (unique when t = π/B):
phi_shift = phi_qpe if phi_qpe <= 0.5 else phi_qpe - 1.0  # in [-1/2, 1/2)
E_from_qpe = -2.0 * B * phi_shift

print(f'Energy from QPE + bound unwrapping: E_QPE = {E_from_qpe}')
print(f'VQE energy reference:             E_VQE = {E_vqe}')
print(f'Delta:                             {E_from_qpe - E_vqe}')


Using StatevectorSampler backend.


QPE most-likely phase (normalized) phi = 0.03515625
QPE most-likely angle theta = 2π phi = 0.22089323345553233 rad
QPE theta mapped to (-π, π] : 0.22089323345553247 rad
Overlap angle arg(<psi|U(t)|psi>) : 0.22619881018233262 rad
Energy from QPE + bound unwrapping: E_QPE = -0.703125
VQE energy reference:             E_VQE = -0.8284271058048679
Delta:                             0.1253021058048679


## Segment 3 — HamiltonianPhaseEstimation energy recovery (scaled QPE)

Qiskit’s HamiltonianPhaseEstimation uses a scale factor \(s=\pi/B\) (with \(B\) a bound on \(|E|\)) and the unitary
\[
U_s := e^{+i s H}.
\]

For an eigenpair \(H\lvert E\rangle=E\lvert E\rangle\),
\[
\begin{aligned}
U_s\lvert E\rangle
&=e^{+isH}\lvert E\rangle\\
&=e^{+isE}\lvert E\rangle\\
&=e^{2\pi i\varphi}\lvert E\rangle,\\
\varphi &\equiv \frac{sE}{2\pi}\;(\mathrm{mod}\;1)
=\frac{E}{2B}\;(\mathrm{mod}\;1),\\
E &= 2B\,\tilde\varphi,\qquad
\tilde\varphi:=\begin{cases}
\varphi, & \varphi\le 1/2,\\
\varphi-1, & \varphi>1/2.
\end{cases}
\end{aligned}
\]

If the Hamiltonian contains an identity component \(c_0 I\), Qiskit removes it internally (identity is a global phase) and adds \(c_0\) back to the reported eigenvalue.


In [5]:
# --- Segment 3 code: HamiltonianPhaseEstimation (scaled QPE) to recover energy ---

# Use a slightly-inflated bound to avoid edge collisions at ±B.
B_hpe = 1.01 * B

evolution = SuzukiTrotter(order=2, reps=1)

hpe = HamiltonianPhaseEstimation(
    num_evaluation_qubits=m,
    sampler=sampler,
    transpiler=pass_manager,
)

hpe_res = hpe.estimate(
    hamiltonian=H_qubit,
    state_preparation=state_prep,
    evolution=evolution,
    bound=B_hpe,
)

phi_hpe = float(hpe_res.phase)  # most-likely phase in [0,1)
E_hpe = float(np.real(hpe_res.most_likely_eigenvalue))

print(f'HPE most-likely phase phi_HPE = {phi_hpe}')
print(f'HPE energy E_HPE = {E_hpe}')
print(f'VQE energy E_VQE = {E_vqe}')
print(f'Delta (HPE - VQE) = {E_hpe - E_vqe}')

# Manual decoding (mirrors PhaseEstimationScale.scale_phase) for transparency.
# Identity-term bookkeeping:
id_label = "I" * H_qubit.num_qubits
id_coeff = 0.0
for pauli_label, coeff in H_qubit.to_list():
    if pauli_label == id_label:
        id_coeff += np.real(coeff)

scale = PhaseEstimationScale(B_hpe)
E_manual = scale.scale_phase(phi_hpe, id_coefficient=id_coeff)

print(f'Identity coefficient (from SparsePauliOp) c0 = {id_coeff}')
print(f'Manual decode (with id shift):           E_manual = {E_manual}')


HPE most-likely phase phi_HPE = 0.86328125
HPE energy E_HPE = -0.76171875
VQE energy E_VQE = -0.8284271058048679
Delta (HPE - VQE) = 0.06670835580486789
Identity coefficient (from SparsePauliOp) c0 = 2.0
Manual decode (with id shift):           E_manual = -0.76171875


## Segment 4 — Optional exact diagonalization check in the target sector (small qubit counts)

If `H_qubit.num_qubits` is small enough, compare against exact diagonalization restricted to the same particle-number sector used by VQE.


In [6]:
# --- Optional exact diagonalization in the VQE particle-number sector ---

n = H_qubit.num_qubits
dim = 2**n

if dim <= 2**14:  # adjust ceiling as desired
    if 'exact_ground_energy_sector' not in _vqe_ns:
        raise RuntimeError('exact_ground_energy_sector is missing from loaded VQE namespace.')

    E_exact = float(
        _vqe_ns['exact_ground_energy_sector'](
            H_hardcoded,
            num_sites=L,
            num_particles=num_particles,
            indexing=HARDCODED_CFG['indexing'],
        )
    )

    print(
        'Exact sector ground energy '
        f'(indexing={HARDCODED_CFG["indexing"]}, N_up={num_particles[0]}, N_dn={num_particles[1]}) '
        f'E_exact = {E_exact}'
    )
    print(f'VQE  delta (vs sector exact): {E_vqe - E_exact}')
    print(f'HPE  delta (vs sector exact): {E_hpe - E_exact}')
else:
    print(f'Skipping exact diagonalization: dim=2**{n} is too large for this cell.')


Exact sector ground energy (indexing=blocked, N_up=1, N_dn=1) E_exact = -0.8284271247461902
VQE  delta (vs sector exact): 1.894132228841272e-08
HPE  delta (vs sector exact): 0.06670837474619018
